# Requete avec les paramètres du jeu de test

In [15]:
#cells pour préparer les requêtes
import urllib.parse

domain = "51.91.251.0"
port = 3000
host = f"http://{domain}:{port}"
path = lambda x: urllib.parse.urljoin(host, x)

In [16]:
import requests

user_id = '545115b6-c8cf-4dbd-9e1c-a8270ea9bd66'
name = 'a'
r = requests.post(path(f'avatars/{user_id}/{name}'))
r

<Response [409]>

In [17]:
r = requests.get(path(f"avatars/{user_id}"))
for avatar in r.json():
    print(avatar['id'], avatar['name'])

6054 max-first-avatar
6055 max-second-avatar
71508 max-third-avatar
71509 max-4th-avatar
111612 max-5th-avatar
303616 test
303617 a
303615 max-f-avatar


In [19]:
import numpy as np
import matplotlib.pyplot as plt
import plotly
import pandas as pd

In [20]:
#importation du jeu test
hotels = pd.read_csv('features_hotels.csv', index_col=['hotel_id', 'city'])
Xtest = pd.read_csv("test_set.csv",index_col =0)
Xtest = Xtest.join(hotels, on=['hotel_id', 'city'])
Xtest

,order_requests,city,date,language,mobile,avatar_id,hotel_id,stock,group,brand,parking,pool,children_policy
index,,,,,,,,,,,,,
0,1,vilnius,21,romanian,0,1,161,46,Boss Western,J.Halliday Inn,1,0,0
1,1,vilnius,21,romanian,0,1,187,32,Accar Hotels,Marcure,1,1,0
2,1,vilnius,21,romanian,0,1,279,12,Independant,Independant,1,0,0
3,1,vilnius,21,romanian,0,1,395,10,Accar Hotels,Ibas,0,0,0
4,1,vilnius,21,romanian,0,1,488,42,Accar Hotels,Safitel,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6639,843,rome,5,irish,0,794,987,1,Accar Hotels,Ibas,1,0,0
6640,844,vienna,1,irish,1,794,26,1,Accar Hotels,Marcure,1,0,0
6641,844,vienna,1,irish,1,794,263,0,Boss Western,Boss Western,1,0,0


In [21]:
#transformation des colonnes intéressantes en liste de dictionnaires, ordonné en dates décroissantes
Xrequete = Xtest[["city","date","language","mobile"]]
Xrequete = Xrequete.drop_duplicates()
Xrequete = Xrequete.sort_values(by="date",ascending = False)
Xrequete["avatar_name"]="test"
Xrequete =Xrequete[["avatar_name","language","city","date","mobile"]]
Requete = Xrequete.to_dict(orient='records')

In [22]:
#Requetes
R = []
for params in Requete:
    r = requests.get(path(f"pricing/{user_id}"), params=params)
    r.json()
    R+= [r]

In [23]:
#Creation du tableau avec les résultats
pricing_requests = []

for r in R:
    pricing_requests.append(
        pd.DataFrame(r.json()['prices']).assign(**r.json()['request'])
    )

pricing_requests = pd.concat(pricing_requests)

In [24]:
# ajouts des autres features
pricing_requests = pricing_requests.join(hotels, on=['hotel_id', 'city'])

In [26]:
#Le jeu de test de Noel :'( vivement la fin du GMM ...
pricing_requests.to_csv('Xtrain24_12.csv')